# Aufgabe 1: Rucksackproblem

Der große Vorteil der dynamischen Programmierung ist die Wiederverwendbarkeit bereits berechneter Werte. Also verwenden wir einen iterativen Ansatz, in dem die Tabelle Zeile für Zeile aufgebaut wird, um groß genug für das abgefragte Gewicht zu werden.

Als erstes implementieren wir also die Funktion, die zu einer bereits bestehenden Tabelle eine weitere Zeile hinzufügt. Hierbei gehen wir davon aus, dass die Tabelle eine Spalte mehr hat als es zu betrachtende Gegenstände gibt, und dass die eingegebene Tabelle mit denselben Gegenständen erzeugt wurde, wie an diese Funktion übergeben werden.

Die Gewichte und Werte der Gegenstände werden in unabhängigen Arrays gespeichert.

In [1]:
def add_row (table, weights, values):
    """Return a table with one more row than and the same contents as the given table.
    Compute the added row as per the dynamic programming paradigm for the knapsack problem."""
    # determine how far we need to iterate in the table
    max_i = min(len(weights), len(values)) + 1
    # the new weight is exactly the amount of rows in the table
    g = len(table)
    # adding the new row
    table = table + [[0 for _ in range(max_i)]]
    # and filling it
    for i in range(1, max_i):
        w = weights[i-1]  # the i-th item is saved in the i-1-th position in these arrays
        # in the following we're always operating on the last row
        if g-w < 0:
            table[-1][i] = table[-1][i-1]
        else:
            # same consideration for `values` as in `weights` above
            table[-1][i] = max(table[-1][i-1], values[i-1] + table[g-w][i-1])
    return table

Nachdem wir die Tabelle generiert haben können wir die Werte auslesen.

In [2]:
def backtrack (table, weights, weight):
    """Return the list of 1-indexed items with maximum cumulative value
    such that the cumulative weight of the items is less or equal to the specified weight."""
    result = []
    # assume that the rows have all the same length
    for i in range(len(table[0]) - 1, 0, -1):
        if table[weight][i] != table[weight][i-1]:
            # again, `weights` is shifted by one relative to the rows in the table
            weight = weight - weights[i-1]
            result.append(i)
    return result

In der Praxis werden wir wohl am ehesten eine optimale Auswahl von Gegenständen für ein gegebenes Gewicht wissen wollen, ohne dafür per Hand erst die Tabelle erzeugen zu müssen. Natürlich wollen wir eine bereits vorhandene Tabelle weiter verwenden können. Die nächste Funktion nimmt also eine Tabelle entgegen, erweitert sie nach Bedarf und extrahiert dann das gewünschte Ergebnis. Zurückgegeben wird das Ergebnis sowie die erweiterte Tabelle für die zukünftige Benutzung.

In [3]:
def backpack (table, weights, values, weight):
    """Return solution to knapsack problem with (partially) precomputed table
    as well as updated table."""
    # add the missing rows
    for _ in range(1 + weight - len(table)):
        table = add_row(table, weights, values)
    return backtrack(table, weights, weight), table

Eine Funktion zum Erstellen einer grundsätzlichen Tabelle für eine gegebene Menge von Gegenständen:

In [4]:
def make_table (weights, values):
    """Return table with one row for use in dynamic programming solution of the knapsack problem."""
    return [[0 for _ in range(1 + min(len(values), len(weights)))]]

Zum Testen:

In [5]:
# define a function for printing tables
def print_table (table):
    """Print table with "|" delimiters between columns."""
    for row in table:
        format_row = "| {:>4} " * len(row) + "|"
        print ( format_row.format(*row) )

weights = [5, 4, 6, 3]
values = [10, 40, 30, 50]
table = make_table(weights, values)

# define a function working on the test data
def backpack_test (weight):
    """Compute and display table and solution for knapsack problem with given items for the specified weight."""
    global weights, values, table
    result, table = backpack(table, weights, values, weight)
    print_table ( table )
    print ( result )

backpack_test(7)

|    0 |    0 |    0 |    0 |    0 |
|    0 |    0 |    0 |    0 |    0 |
|    0 |    0 |    0 |    0 |    0 |
|    0 |    0 |    0 |    0 |   50 |
|    0 |    0 |   40 |   40 |   50 |
|    0 |   10 |   40 |   40 |   50 |
|    0 |   10 |   40 |   40 |   50 |
|    0 |   10 |   40 |   40 |   90 |
[4, 2]


und um die Wiederverwendbarkeit der Tabelle zu demonstrieren:

In [6]:
backpack_test(17)

|    0 |    0 |    0 |    0 |    0 |
|    0 |    0 |    0 |    0 |    0 |
|    0 |    0 |    0 |    0 |    0 |
|    0 |    0 |    0 |    0 |   50 |
|    0 |    0 |   40 |   40 |   50 |
|    0 |   10 |   40 |   40 |   50 |
|    0 |   10 |   40 |   40 |   50 |
|    0 |   10 |   40 |   40 |   90 |
|    0 |   10 |   40 |   40 |   90 |
|    0 |   10 |   50 |   50 |   90 |
|    0 |   10 |   50 |   70 |   90 |
|    0 |   10 |   50 |   70 |   90 |
|    0 |   10 |   50 |   70 |  100 |
|    0 |   10 |   50 |   70 |  120 |
|    0 |   10 |   50 |   70 |  120 |
|    0 |   10 |   50 |   80 |  120 |
|    0 |   10 |   50 |   80 |  120 |
|    0 |   10 |   50 |   80 |  120 |
[4, 3, 2]


# Aufgabe 2
## a)

Wir suchen die bedingte Übergangsmatrix $P$ für das Matchstick Game, wenn der Agent, wenn möglich, immer die Aktion selbst wählt und die Umgebung zufällig gleichverteilt 1,2 oder 3 als Aktion wählt.

Wie in der Vorlesung beginnt das Spiel mit 21 Stäbchen. 

Es werden in jeder Runde immer 3 (2+1),4 (2+2) oder 5 (2+3) Stäbchen weniger, jeweils mit Wahrscheinlichkeit $\frac{1}{3}$.

Es gibt wieder 22 Zustände: Im $n$-ten Zustand sind noch genau $n$ Stäbchen verfügbar ($n \in \{0, 1, ..., 21\}).

Dabei ist Zustand 20 und 19 aber nicht erreichbar, da vom Startzustand (21) aus durch die vorgegebene Policy ($A_t = 2$) mindestens 3 Stäbchen genommen werden.
Außerdem kann von keinem Zustand aus durch eine legale Aktion Zustand 21 erreicht werden.

Der Zustand $n$ wird dargestellt als ein 22-Tupel mit einer $1$ an Stelle $n$ und sonst nur Nullen. So kann mit $s \cdot P$ die Wahrscheinlichkeit für jeden Zustandsübergang von $s$ nach $s'$ erhalten werden.

Die Matrix $P$ ist also wieder eine $22\times 22$ Matrix.

Betrachten wir den Zustand $s = n$, dann sind mit der gegebenen Policy $A_t = 2$ nur die Folgezustände $n-3$, $n-4$ und $n-5$ möglich, außer es sind nicht mehr genügend Stäbchen vorhanden. Die drei Möglicheiten sind dabei alle gleich wahrscheinlich.

Also ergibt sich folgende Matrix:
$$
P = \left(
\begin{array}{cccccccccccccccccccccc}
 1 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 \\ % 0
 1 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 \\ % 1
 1 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 \\ % 2
 1 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 \\ % 3
 \frac{1}{2} & \frac{1}{2} & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 \\ % 4
 \frac{1}{3} & \frac{1}{3} & \frac{1}{3} & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 \\ % 5
 0 & \frac{1}{3} & \frac{1}{3} & \frac{1}{3} & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 \\ % 6
 0 & 0 & \frac{1}{3} & \frac{1}{3} & \frac{1}{3} & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 \\ % 7
 0 & 0 & 0 & \frac{1}{3} & \frac{1}{3} & \frac{1}{3} & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 \\ % 8
 0 & 0 & 0 & 0 & \frac{1}{3} & \frac{1}{3} & \frac{1}{3} & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 \\ % 9
 0 & 0 & 0 & 0 & 0 & \frac{1}{3} & \frac{1}{3} & \frac{1}{3} & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 \\ % 10
 0 & 0 & 0 & 0 & 0 & 0 & \frac{1}{3} & \frac{1}{3} & \frac{1}{3} & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 \\ % 11
 0 & 0 & 0 & 0 & 0 & 0 & 0 & \frac{1}{3} & \frac{1}{3} & \frac{1}{3} & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 \\ % 12
 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & \frac{1}{3} & \frac{1}{3} & \frac{1}{3} & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 \\ % 13
 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & \frac{1}{3} & \frac{1}{3} & \frac{1}{3} & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 \\ % 14
 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & \frac{1}{3} & \frac{1}{3} & \frac{1}{3} & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 \\ % 15
 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & \frac{1}{3} & \frac{1}{3} & \frac{1}{3} & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 \\ % 16
 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & \frac{1}{3} & \frac{1}{3} & \frac{1}{3} & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 \\ % 17
 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & \frac{1}{3} & \frac{1}{3} & \frac{1}{3} & 0 & 0 & 0 & 0 & 0 & 0 & 0 \\ % 18
 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & \frac{1}{3} & \frac{1}{3} & \frac{1}{3} & 0 & 0 & 0 & 0 & 0 & 0 \\ % 19
 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & \frac{1}{3} & \frac{1}{3} & \frac{1}{3} & 0 & 0 & 0 & 0 & 0 \\ % 20
 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & \frac{1}{3} & \frac{1}{3} & \frac{1}{3} & 0 & 0 & 0 & 0 \\ % 21
 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & \frac{1}{3} & \frac{1}{3} & \frac{1}{3} & 0 & 0 & 0 \\ % 22
\end{array}
\right)
$$

Besonders sind hier die Fälle $0$ bis $4$, die abweichende Wahrscheinlichkeiten beinhalten. Dazu hier die Erklärung:
- In Zustand 4 nimmt der Agent zuerst 2 Stäbchen. Es bleiben also noch 2. Spieler 2 nimmt dann mit gleicher Wahrscheinlichkeit 1 oder 2 Stäbchen, da drei nicht mehr möglich sind. Somit ergibt sich die Wahrscheinlichkeit von jeweils $\frac{1}{2}$ für die Übergänge in Zustand $0$ bzw. $1$.

- In Zustand 3 nimmt der Agent 2 Stäbchen, sodass Spieler 2 nur die Option bleibt das letzte zu nehmen, sodass ein Übergang zu Zustand $0$ garantiert ist.
- In Zustand 2 nimmt der Agent 2 Stäbchen und verliert damit sofort (erreicht Zustand $0$).
- In Zustand 1 kann der Agent keine 2 Stäbchen nehmen, nimmt also nur das eine verbleibende und verliert damit sofort (erreicht Zustand $0$).

Außerdem ist in der Matrix ersichtlich, dass die Zustände $21$, $20$ und $19$ nicht Folgezustände anderer sind. Die vorletzten zwei Zeilen (19 und 20) sind also für die gegebenen Strategien eigentlich nie relevant.

## b)

Wir wollen prüfen ob der Bellman-Erwartungs-Operator $T^\pi$ die Voraussetzungen für den Banachschen Fixpunktsatz erfüllt, also eine Kontraktion ist.

Dazu betrachten wir den metrischen Raum $\mathbb{R}^{22}$ und definieren $T^\pi(v) := R^\pi + \gamma P^\pi v$.
Wir gehen davon aus, dass alle $v$ in einer kompakten Teilmenge von $\mathbb{R}^{22}$ liegen, ein Zustand also nur begrenzt große Wertung hat.

Nun wollen prüfen, ob $T^\pi$ eine $\gamma$-Kontraktion ist.

### Beweis:

- $R^\pi \in \mathbb{R}^{22}$ ist ein konstanter Vektor.
- $P^\pi$ ist wie in a) definiert die Zustands-Übergangsmatrix.
- $\gamma \in [0,1]$

Seien $a,b \in \mathbb{R}^{22}$ mit $d(a,b) = \sqrt{(b_1 - a_1)^2 + ... (b_{22}-a_{22})^2}$.

Zu zeigen ist nun also dass $d(T^\pi(a), T^\pi(b)) \leq \gamma \, d(a,b)$ gilt.

Einsetzen Definition von $T^\pi$ ergibt:
$$
\begin{align}
d(T^\pi(a), T^\pi(b)) &= \big| T^\pi(b)-T^\pi(a) \big|                                          \\
                      &= \big| ( R^\pi + \gamma P^\pi b )  -  ( R^\pi + \gamma P^\pi a ) \big|  \\
                      &= \big|  R^\pi - R^\pi  +  \gamma P^\pi b - \gamma P^\pi a  \big|        \\
                      &= \big|  \gamma (P^\pi b - P^\pi a)  \big|                               \\
                      &\overset{\gamma \, \geq \, 0}{=} \gamma \, \big| P^\pi b - P^\pi a \big| \\
                      &= \gamma \, \big| P^\pi (b - a) \big|                                    \\
                      &\not\leq \gamma \, |b-a|                                                 \\
                      &= \gamma \, d(a,b)
\end{align}
$$

Diese Ungleichung gilt nicht. Betrachte dazu folgendes Gegenbeispiel:


Sei $a = 0 \in \mathbb{R}^{22}$ der Nullvektor und $e_1 = (1,0,0,...,0) \in \mathbb{R}^{22}$ der Einheitsvektor.

Beide Vektoren stellen denkbare Bewertungsfunktionen $V$ dar.

Dann gilt aber $P^\pi\,(e_1-a) = P^\pi e_1 = (1,1,1,1,\frac{1}{2}, \frac{1}{3}, 0,0, ...,0)$. Damit gilt offensichtlich $|P^\pi e_1| > |e_1|$, sodass die Ungleichung widerlegt ist.

Daher ist die Funktion $T^\pi$ **keine** Kontraktion.